<a href="https://colab.research.google.com/github/cbologa/public_databases/blob/main/R/DrugCentral_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load required packages

In [ ]:
if(!require(pacman)) install.packages("pacman")
pacman::p_load(RPostgreSQL)

Connect to database

In [4]:
con <- dbConnect(PostgreSQL(), dbname = "drugcentral", host="unmtid-dbs.net", port=5433, user="drugman", password="dosage")

Show all tables in the database

In [ ]:
dbListTables(con)

There are also 4 dynamic tables (views) that are not be listed above:

* faers_top
* ob_exclusivity_view
* ob_patent_view
* omop_relationship_doid_view

Show fields in a table/view

In [ ]:
dbGetQuery(con,"
select
  *
from
  information_schema.columns
where
  table_schema = 'public' and
  table_name = 'omop_relationship_doid_view';
")

Extract all drugs and their mechanism of action (MoA) on human targets.

In [ ]:
df1 <- dbGetQuery(con, "
select 
  struct_id,
	structures.name,
	target_name,
	target_class,
	accession,
	gene,
	swissprot,
	act_value,
	act_type,
	act_source,
	act_source_url,
	moa_source,
	moa_source_url 
from 
  act_table_full
	join structures on act_table_full.struct_id = structures.id
where
  moa = 1 and organism = 'Homo sapiens'
")

#save results to file
write.csv(df1, "dc_moa.csv", row.names=FALSE)

# Show results
df1

Get all DOID-mapped indications from omop_relationship_doid_view

In [ ]:
df2 <- dbGetQuery(con, "
select
  *
from
  omop_relationship_doid_view
where
  relationship_name = 'indication' and
  doid is not null
order by
  concept_name
")

#save results to file
write.csv(df2, "dc_doid.csv", row.names=FALSE)

# Show results
df2

Close database connection

In [ ]:
dbDisconnect(con)